In [1]:
import pandas as pd

from tableone import TableOne

In [2]:
data = pd.read_gbq("""
WITH ed AS (
    SELECT
          stays.hadm_id
        , SUM(TIMESTAMP_DIFF(stays.outtime, stays.intime, HOUR)) AS ed_los
        , MAX(triage.acuity) AS ed_acuity
    FROM `lcp-internal.mimic_ed.edstays` stays
    LEFT JOIN `lcp-internal.mimic_ed.triage` triage
        ON stays.stay_id = triage.stay_id
    LEFT JOIN `lcp-internal.mimic_ed.diagnosis` diag
        ON stays.stay_id = diag.stay_id
        AND diag.seq_num = 1
    GROUP BY stays.hadm_id
)
SELECT
      pat.subject_id
    , adm.hadm_id
    , icu.stay_id
    , CASE
        WHEN FIRST_VALUE(icu.stay_id) OVER icustay_window = icu.stay_id THEN 1
        ELSE 0
      END AS pat_count
    , CASE
        WHEN FIRST_VALUE(adm.hadm_id) OVER hadm_window = adm.hadm_id THEN 1
        ELSE 0
      END AS hadm_count
    , pat.anchor_age + (EXTRACT(YEAR FROM icu.intime) - pat.anchor_year) AS age
    , pat.gender
    , adm.insurance
    , ed.ed_acuity
    , ed.ed_los
    , icu.first_careunit
    , icu.los AS icu_los
    , TIMESTAMP_DIFF(adm.dischtime, adm.admittime, HOUR) / 24 AS hosp_los
    , adm.hospital_expire_flag
FROM `lcp-internal.mimic_hosp.patients` pat
INNER JOIN `lcp-internal.mimic_hosp.admissions` adm
    ON pat.subject_id = adm.subject_id
INNER JOIN `lcp-internal.mimic_icu.icustays` icu
    ON adm.hadm_id = icu.hadm_id
LEFT JOIN ed
    ON adm.hadm_id = ed.hadm_id
WINDOW hadm_window AS (PARTITION BY pat.subject_id ORDER BY adm.admittime)
     , icustay_window AS (PARTITION BY pat.subject_id ORDER BY icu.intime)
""", "lcp-internal")

In [3]:
int_cols = data.dtypes.values=="Int64"
data.loc[:, int_cols] = data.loc[:, int_cols].astype(float)
data.loc[:, int_cols] = data.loc[:, int_cols].astype(int, errors="ignore")

In [4]:
columns = [
    "pat_count", "hadm_count",
    "age", "gender", "insurance",
    "first_careunit",
    "ed_acuity", "ed_los",
    "icu_los", "hosp_los"
]

categorical = [
    "pat_count", "hadm_count",
    "gender", "insurance",
    "first_careunit",
    "ed_acuity",
]

order = {
    "pat_count": [1, 0], "hadm_count": [1, 0],
    "gender": ["F", "M"]
}

limit = {
    "pat_count": 1, "hadm_count": 1,
    "gender": 1
}

rename = {
    "pat_count": "Distinct patients", "hadm_count": "Distinct hospitalizations",
    "age": "Age", "gender": "Gender", "insurance": "Insurance",
    "first_careunit": "First ICU stay, unit type",
    "ed_acuity": "ED acuity", "ed_los": "ED length of stay",
    "icu_los": "ICU length of stay", "hosp_los": "Hospital length of stay"
}

table = TableOne(data, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename,
                 groupby="hospital_expire_flag")
table

/opt/homebrew/Caskroom/miniconda/base/envs/sandbox/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/opt/homebrew/Caskroom/miniconda/base/envs/sandbox/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/opt/homebrew/Caskroom/miniconda/base/envs/sandbox/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(l

Grouped by hospital_expire_flag                                         
                                                                                                            Missing       Overall             0            1
n                                                                                                                           76943         68149         8794
Distinct patients, n (%)           1                                                                              0  53569 (69.6)  48065 (70.5)  5504 (62.6)
Distinct hospitalizations, n (%)   1                                                                              0  58881 (76.5)  52403 (76.9)  6478 (73.7)
Age, mean (SD)                                                                                                    0   64.7 (16.9)   63.9 (16.9)  70.5 (15.2)
Gender, n (%)                      F                                                                              0  34091 (44.3)  30106 (44.2)  3985 (45.3)
Insurance, n (%)                   Medicaid                                                                       0    5825 (7.6)    5292 (7.8)    533 (6.1)
                                   Medicare                                                                          34740 (45.2)  29995 (44.0)  4745 (54.0)
                                   Other                                                                             36378 (47.3)  32862 (48.2)  3516 (40.0)
First ICU stay, unit type, n (%)   Cardiac Vascular Intensive Care Unit (CVICU)                                   0  12148 (15.8)  11633 (17.1)    515 (5.9)
                                   Coronary Care Unit (CCU)                                                           8730 (11.3)   7602 (11.2)  1128 (12.8)
                                   Medical Intensive Care Unit (MICU)                                                16705 (21.7)  14237 (20.9)  2468 (28.1)
                                   Medical/Surgical Intensive Care Unit (MICU/SICU)                                  13404 (17.4)  11440 (16.8)  1964 (22.3)
                                   Neuro Intermediate                                                                  2138 (2.8)    2098 (3.1)     40 (0.5)
                                   Neuro Stepdown                                                                      1074 (1.4)    1051 (1.5)     23 (0.3)
                                   Neuro Surgical Intensive Care Unit (Neuro SICU)                                     1840 (2.4)    1539 (2.3)    301 (3.4)
                                   Surgical Intensive Care Unit (SICU)                                               11753 (15.3)  10355 (15.2)  1398 (15.9)
                                   Trauma SICU (TSICU)                                                                9151 (11.9)   8194 (12.0)   957 (10.9)
ED acuity, n (%)                   1.000000000                                                                43686   9984 (30.0)   8379 (28.7)  1605 (39.5)
                                   2.000000000                                                                       18242 (54.9)  16294 (55.8)  1948 (47.9)
                                   3.000000000                                                                        5009 (15.1)   4499 (15.4)   510 (12.6)
                                   4.000000000                                                                           21 (0.1)      21 (0.1)             
                                   5.000000000                                                                            1 (0.0)       1 (0.0)             
ED length of stay, mean (SD)                                                                                  39934     5.6 (4.6)     5.6 (3.8)    5.5 (8.0)
ICU length of stay, mean (SD)                                                                                     0     3.5 (4.9)     3.2 (4.6)    5.0 (6.8)
Ho